In [ ]:
# eduhub_setup.py (or notebook cell)
from pymongo import MongoClient
from pymongo.errors import CollectionInvalid
from datetime import datetime
import re
client = MongoClient("mongodb://localhost:27017/")
db = client["eduhub_db"]

In [2]:
# eduhub_setup.py (or notebook cell)
from pymongo import MongoClient
from pymongo.errors import CollectionInvalid
from datetime import datetime
import re

client = MongoClient("mongodb://localhost:27017/")
db = client["eduhub_db"]

# JSON Schema validators for collections
user_validator = {
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["userId", "email", "firstName", "lastName", "role", "dateJoined", "isActive"],
        "properties": {
            "userId": {"bsonType": "string"},
            "email": {"bsonType": "string", "pattern": r"^[^@ \t\r\n]+@[^@ \t\r\n]+\.[^@ \t\r\n]+$"},
            "firstName": {"bsonType": "string"},
            "lastName": {"bsonType": "string"},
            "role": {"enum": ["student", "instructor", "admin"]},
            "dateJoined": {"bsonType": "date"},
            "profile": {
                "bsonType": "object",
                "properties": {
                    "bio": {"bsonType": "string"},
                    "avatar": {"bsonType": "string"},
                    "skills": {"bsonType": "array", "items": {"bsonType": "string"}}
                }
            },
            "isActive": {"bsonType": "bool"}
        }
    }
}

course_validator = {
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["courseId", "title", "instructorId", "createdAt", "isPublished"],
        "properties": {
            "courseId": {"bsonType": "string"},
            "title": {"bsonType": "string"},
            "description": {"bsonType": "string"},
            "instructorId": {"bsonType": "string"},
            "category": {"bsonType": "string"},
            "level": {"enum": ["beginner", "intermediate", "advanced"]},
            "duration": {"bsonType": "double"},
            "price": {"bsonType": ["double", "int"]},
            "tags": {"bsonType": "array", "items": {"bsonType": "string"}},
            "createdAt": {"bsonType": "date"},
            "updatedAt": {"bsonType": ["date", "null"]},
            "isPublished": {"bsonType": "bool"},
            "rating": {"bsonType": ["double", "null"], "minimum": 0, "maximum": 5}
        }
    }
}

enrollment_validator = {
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["enrollmentId", "studentId", "courseId", "enrolledAt", "status"],
        "properties": {
            "enrollmentId": {"bsonType": "string"},
            "studentId": {"bsonType": "string"},
            "courseId": {"bsonType": "string"},
            "enrolledAt": {"bsonType": "date"},
            "status": {"enum": ["in_progress", "completed", "dropped"]},
            "progress": {"bsonType": "double", "minimum": 0, "maximum": 100}
        }
    }
}

lesson_validator = {
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["lessonId", "courseId", "title", "order"],
        "properties": {
            "lessonId": {"bsonType": "string"},
            "courseId": {"bsonType": "string"},
            "title": {"bsonType": "string"},
            "content": {"bsonType": "string"},
            "durationMinutes": {"bsonType": ["int","double"]},
            "order": {"bsonType": "int"}
        }
    }
}

assignment_validator = {
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["assignmentId", "courseId", "title", "dueDate"],
        "properties": {
            "assignmentId": {"bsonType": "string"},
            "courseId": {"bsonType": "string"},
            "title": {"bsonType": "string"},
            "description": {"bsonType": "string"},
            "dueDate": {"bsonType": "date"},
            "maxPoints": {"bsonType": "int"}
        }
    }
}

submission_validator = {
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["submissionId", "assignmentId", "studentId", "submittedAt"],
        "properties": {
            "submissionId": {"bsonType": "string"},
            "assignmentId": {"bsonType": "string"},
            "studentId": {"bsonType": "string"},
            "submittedAt": {"bsonType": "date"},
            "grade": {"bsonType": ["double", "null"]},
            "feedback": {"bsonType": ["string", "null"]},
            "files": {"bsonType": ["array", "null"], "items": {"bsonType": "string"}}
        }
    }
}

# Create collections with validators (if they don't exist)
validators = {
    "users": user_validator,
    "courses": course_validator,
    "enrollments": enrollment_validator,
    "lessons": lesson_validator,
    "assignments": assignment_validator,
    "submissions": submission_validator
}

for coll_name, v in validators.items():
    try:
        db.create_collection(coll_name)
    except CollectionInvalid:
        pass
    db.command("collMod", coll_name, validator=v, validationLevel="moderate")


In [4]:
# sample_data_generator.py
from pymongo import MongoClient
from datetime import datetime, timedelta
import random
import json
from faker import Faker  # pip install faker
fake = Faker()

client = MongoClient("mongodb://localhost:27017/")
db = client["eduhub_db"]

def gen_users(n_students=15, n_instructors=5):
    users = []
    # instructors
    for i in range(n_instructors):
        uid = f"instr_{100+i}"
        users.append({
            "userId": uid,
            "email": f"{uid}@example.com",
            "firstName": fake.first_name(),
            "lastName": fake.last_name(),
            "role": "instructor",
            "dateJoined": fake.date_time_between(start_date='-2y', end_date='now'),
            "profile": {"bio": fake.sentence(nb_words=10), "avatar": "", "skills": fake.words(nb=3)},
            "isActive": True
        })
    # students
    for i in range(n_students):
        uid = f"stud_{1000+i}"
        users.append({
            "userId": uid,
            "email": f"{uid}@example.com",
            "firstName": fake.first_name(),
            "lastName": fake.last_name(),
            "role": "student",
            "dateJoined": fake.date_time_between(start_date='-1y', end_date='now'),
            "profile": {"bio": fake.sentence(nb_words=8), "avatar": "", "skills": fake.words(nb=2)},
            "isActive": True
        })
    return users

def gen_courses(instr_ids, n=8):
    cats = ["programming","data-science","design","business","finance","language"]
    levels = ["beginner","intermediate","advanced"]
    courses=[]
    for i in range(n):
        cid = f"course_{200+i}"
        instr = random.choice(instr_ids)
        created = fake.date_time_between(start_date='-2y', end_date='now')
        courses.append({
            "courseId": cid,
            "title": fake.sentence(nb_words=4).rstrip('.'),
            "description": fake.paragraph(nb_sentences=2),
            "instructorId": instr,
            "category": random.choice(cats),
            "level": random.choice(levels),
            "duration": round(random.uniform(1.0, 40.0), 1),
            "price": round(random.choice([0, 19.99, 49.99, 99.99, 149.99, 199.99]),2),
            "tags": fake.words(nb=3),
            "createdAt": created,
            "updatedAt": None,
            "isPublished": random.choice([True, False]),
            "rating": round(random.uniform(3.0,5.0),1)
        })
    return courses

def populate():
    users = gen_users()
    db.users.delete_many({})
    db.courses.delete_many({})
    db.enrollments.delete_many({})
    db.lessons.delete_many({})
    db.assignments.delete_many({})
    db.submissions.delete_many({})

    db.users.insert_many(users)
    instructor_ids = [u["userId"] for u in users if u["role"]=="instructor"]
    student_ids = [u["userId"] for u in users if u["role"]=="student"]

    courses = gen_courses(instructor_ids, 8)
    db.courses.insert_many(courses)

    # enrollments sample
    enrollments=[]
    ecount=0
    for student in student_ids[:15]:
        chosen = random.sample(courses, k=random.randint(1,3))
        for c in chosen:
            ecount += 1
            enrollments.append({
                "enrollmentId": f"enr_{ecount}",
                "studentId": student,
                "courseId": c["courseId"],
                "enrolledAt": fake.date_time_between(start_date=c["createdAt"], end_date='now'),
                "status": random.choice(["in_progress","completed","dropped"]),
                "progress": round(random.uniform(0,100),1)
            })
    db.enrollments.insert_many(enrollments)

    # lessons: 25 lessons across courses
    lessons = []
    lid=0
    for c in courses:
        n_lessons = random.randint(2,6)
        for o in range(1,n_lessons+1):
            lid += 1
            lessons.append({
                "lessonId": f"lesson_{lid}",
                "courseId": c["courseId"],
                "title": f"{c['title']} - Lesson {o}",
                "content": fake.paragraph(nb_sentences=3),
                "durationMinutes": random.randint(5,60),
                "order": o
            })
    db.lessons.insert_many(lessons)

    # assignments: 10
    assignments=[]
    aid=0
    for c in courses[:6]:
        for _ in range(random.randint(0,2)):
            aid += 1
            assignments.append({
                "assignmentId": f"assign_{aid}",
                "courseId": c["courseId"],
                "title": f"{c['title']} Assignment {aid}",
                "description": fake.paragraph(nb_sentences=2),
                "dueDate": fake.date_time_between(start_date='now', end_date='+30d'),
                "maxPoints": 100
            })
    db.assignments.insert_many(assignments)

    # submissions: 12
    submissions=[]
    sid=0
    for a in assignments[:12]:
        for stud in random.sample(student_ids, k=min(3,len(student_ids))):
            sid+=1
            submissions.append({
                "submissionId": f"sub_{sid}",
                "assignmentId": a["assignmentId"],
                "studentId": stud,
                "submittedAt": fake.date_time_between(start_date=a["dueDate"] - timedelta(days=10), end_date=a["dueDate"]),
                "grade": round(random.uniform(50,100),1),
                "feedback": fake.sentence(nb_words=6),
                "files": []
            })
    db.submissions.insert_many(submissions)

if __name__ == "__main__":
    populate()
    print("Sample data populated.")


Sample data populated.


In [6]:
from pymongo import MongoClient
from datetime import datetime

client = MongoClient("mongodb://localhost:27017/")
db = client["eduhub_db"]

# Add a new student user
new_user = {
    "userId": "stud_9999",
    "email": "newstudent@example.com",
    "firstName": "Samuel",
    "lastName": "Okoro",
    "role": "student",
    "dateJoined": datetime.utcnow(),
    "profile": {"bio": "New student", "avatar": "", "skills": []},
    "isActive": True
}
db.users.insert_one(new_user)

C:\Users\SYLVESTER\AppData\Local\Temp\ipykernel_3720\1955656641.py:14: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "dateJoined": datetime.utcnow(),


InsertOneResult(ObjectId('68e1c70a2cb9762fe4258e56'), acknowledged=True)

In [9]:
from pymongo import MongoClient
from datetime import datetime

client = MongoClient("mongodb://localhost:27017/")
db = client["eduhub_db"]
# Create a new course
new_course = {
    "courseId": "course_999",
    "title": "Automated Testing",
    "description": "Unit testing and integration testing.",
    "instructorId": "instr_100",
    "category": "programming",
    "level": "intermediate",
    "duration": 8.0,
    "price": 79.99,
    "tags": ["testing","ci"],
    "createdAt": datetime.utcnow(),
    "updatedAt": None,
    "isPublished": False
}
db.courses.insert_one(new_course)



C:\Users\SYLVESTER\AppData\Local\Temp\ipykernel_3720\294335966.py:17: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "createdAt": datetime.utcnow(),


InsertOneResult(ObjectId('68e1c8d22cb9762fe4258e5c'), acknowledged=True)

In [10]:
from pymongo import MongoClient
from datetime import datetime

client = MongoClient("mongodb://localhost:27017/")
db = client["eduhub_db"]
# Enroll a student in a course
enrollment = {
    "enrollmentId": "enr_999",
    "studentId": "stud_9999",
    "courseId": "course_999",
    "enrolledAt": datetime.utcnow(),
    "status": "in_progress",
    "progress": 0.0
}
db.enrollments.insert_one(enrollment)


C:\Users\SYLVESTER\AppData\Local\Temp\ipykernel_3720\3364821550.py:11: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "enrolledAt": datetime.utcnow(),


InsertOneResult(ObjectId('68e1c9362cb9762fe4258e5e'), acknowledged=True)

In [11]:
from pymongo import MongoClient
from datetime import datetime

client = MongoClient("mongodb://localhost:27017/")
db = client["eduhub_db"]
# Add a new lesson to existing course
lesson = {
    "lessonId": "lesson_999",
    "courseId": "course_999",
    "title": "Intro to testing",
    "content": "Lesson content here",
    "durationMinutes": 30,
    "order": 1
}
db.lessons.insert_one(lesson)

InsertOneResult(ObjectId('68e1c96d2cb9762fe4258e60'), acknowledged=True)

In [16]:
active_students = list(db.users.find({"role":"student","isActive":True}))
print(active_students)

[{'_id': ObjectId('68e1c6332cb9762fe4258de2'), 'userId': 'stud_1000', 'email': 'stud_1000@example.com', 'firstName': 'Craig', 'lastName': 'Butler', 'role': 'student', 'dateJoined': datetime.datetime(2025, 3, 8, 22, 17, 22), 'profile': {'bio': 'Turn director affect agreement live difference very raise.', 'avatar': '', 'skills': ['month', 'road']}, 'isActive': True}, {'_id': ObjectId('68e1c6332cb9762fe4258de3'), 'userId': 'stud_1001', 'email': 'stud_1001@example.com', 'firstName': 'Gregory', 'lastName': 'Roberts', 'role': 'student', 'dateJoined': datetime.datetime(2025, 1, 9, 16, 49, 43), 'profile': {'bio': 'Growth attorney which own effect arrive seat same.', 'avatar': '', 'skills': ['community', 'food']}, 'isActive': True}, {'_id': ObjectId('68e1c6332cb9762fe4258de4'), 'userId': 'stud_1002', 'email': 'stud_1002@example.com', 'firstName': 'Pamela', 'lastName': 'Hernandez', 'role': 'student', 'dateJoined': datetime.datetime(2025, 1, 28, 21, 1, 42), 'profile': {'bio': 'Without over nearly

In [15]:
pipeline = [
    {"$match": {"courseId": "course_999"}},
    {"$lookup": {
        "from": "users",
        "localField": "instructorId",
        "foreignField": "userId",
        "as": "instructor"
    }},
    {"$unwind": {"path": "$instructor", "preserveNullAndEmptyArrays": True}}
]
course_with_instructor = list(db.courses.aggregate(pipeline))
print(course_with_instructor)

[{'_id': ObjectId('68e1c8d22cb9762fe4258e5c'), 'courseId': 'course_999', 'title': 'Automated Testing', 'description': 'Unit testing and integration testing.', 'instructorId': 'instr_100', 'category': 'programming', 'level': 'intermediate', 'duration': 8.0, 'price': 79.99, 'tags': ['testing', 'ci'], 'createdAt': datetime.datetime(2025, 10, 5, 1, 24, 34, 579000), 'updatedAt': None, 'isPublished': False, 'instructor': {'_id': ObjectId('68e1c6332cb9762fe4258ddd'), 'userId': 'instr_100', 'email': 'instr_100@example.com', 'firstName': 'Brittney', 'lastName': 'Sloan', 'role': 'instructor', 'dateJoined': datetime.datetime(2024, 9, 20, 6, 59, 17), 'profile': {'bio': 'Minute near remain unit argue mean identify case.', 'avatar': '', 'skills': ['new', 'through', 'senior']}, 'isActive': True}}]


In [17]:
enrolled_students = list(db.enrollments.find({"courseId": "course_999"}))
print(enrolled_students)

[{'_id': ObjectId('68e1c9362cb9762fe4258e5e'), 'enrollmentId': 'enr_999', 'studentId': 'stud_9999', 'courseId': 'course_999', 'enrolledAt': datetime.datetime(2025, 10, 5, 1, 26, 14, 343000), 'status': 'in_progress', 'progress': 0.0}]


In [14]:
prog_courses = list(db.courses.find({"category": "programming"}))
print(prog_courses)

[{'_id': ObjectId('68e1c6332cb9762fe4258df6'), 'courseId': 'course_205', 'title': 'Measure number', 'description': 'Health traditional history person season. Page group executive hot computer level wait.', 'instructorId': 'instr_100', 'category': 'programming', 'level': 'beginner', 'duration': 1.1, 'price': 199.99, 'tags': ['him', 'begin', 'know'], 'createdAt': datetime.datetime(2025, 2, 16, 8, 37, 47), 'updatedAt': None, 'isPublished': False, 'rating': 3.6}, {'_id': ObjectId('68e1c6332cb9762fe4258df8'), 'courseId': 'course_207', 'title': 'Remember song', 'description': 'Size very suggest. But one since sound.', 'instructorId': 'instr_103', 'category': 'programming', 'level': 'advanced', 'duration': 10.1, 'price': 99.99, 'tags': ['rate', 'keep', 'let'], 'createdAt': datetime.datetime(2024, 1, 28, 0, 15, 44), 'updatedAt': None, 'isPublished': False, 'rating': 3.3}, {'_id': ObjectId('68e1c8d22cb9762fe4258e5c'), 'courseId': 'course_999', 'title': 'Automated Testing', 'description': 'Unit 

In [19]:
search_term = "python"
courses_found = list(db.courses.find({"title": {"$regex": search_term, "$options": "i"}}))
print(courses_found)

[]


In [20]:
# Update a user's profile information
db.users.update_one({"userId":"stud_9999"}, {"$set":{"profile.bio":"Updated bio","profile.skills":["testing","git"]}})




UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [21]:

# Update assignment grades
db.submissions.update_one({"submissionId":"sub_1"}, {"$set":{"grade":95.0, "feedback":"Excellent"}})


UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [22]:
# Add tags to existing course (append if not present)
db.courses.update_one({"courseId":"course_999"}, {"$addToSet":{"tags": {"$each":["automation","qa"]}}})

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [23]:
# Mark a course as published
db.courses.update_one({"courseId":"course_999"}, {"$set":{"isPublished": True, "updatedAt": datetime.utcnow()}})


C:\Users\SYLVESTER\AppData\Local\Temp\ipykernel_3720\612115014.py:2: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  db.courses.update_one({"courseId":"course_999"}, {"$set":{"isPublished": True, "updatedAt": datetime.utcnow()}})


UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [24]:
# Soft delete user
db.users.update_one({"userId":"stud_9999"}, {"$set":{"isActive": False}})

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [25]:
# Delete an enrollment (hard delete)
db.enrollments.delete_one({"enrollmentId":"enr_999"})

DeleteResult({'n': 1, 'ok': 1.0}, acknowledged=True)

In [26]:
# Remove a lesson from a course
db.lessons.delete_one({"lessonId":"lesson_999"})

DeleteResult({'n': 1, 'ok': 1.0}, acknowledged=True)

In [28]:
# Find courses with price between $50 and $200
courses_price_range = list(db.courses.find({"price": {"$gte": 50, "$lte": 200}}))
print(courses_price_range)

[{'_id': ObjectId('68e1c6332cb9762fe4258df3'), 'courseId': 'course_202', 'title': 'He measure', 'description': 'Free simple sort occur prepare magazine. Method clear daughter number administration.', 'instructorId': 'instr_100', 'category': 'design', 'level': 'advanced', 'duration': 9.5, 'price': 199.99, 'tags': ['whom', 'that', 'week'], 'createdAt': datetime.datetime(2024, 2, 3, 4, 52, 36), 'updatedAt': None, 'isPublished': False, 'rating': 3.6}, {'_id': ObjectId('68e1c6332cb9762fe4258df5'), 'courseId': 'course_204', 'title': 'Player author great', 'description': 'Its partner meeting off. Technology order subject raise.', 'instructorId': 'instr_102', 'category': 'finance', 'level': 'intermediate', 'duration': 21.0, 'price': 199.99, 'tags': ['we', 'resource', 'nearly'], 'createdAt': datetime.datetime(2025, 6, 26, 1, 31, 8), 'updatedAt': None, 'isPublished': False, 'rating': 4.0}, {'_id': ObjectId('68e1c6332cb9762fe4258df6'), 'courseId': 'course_205', 'title': 'Measure number', 'descrip

In [30]:
# Get users who joined in the last 6 months
from datetime import datetime, timedelta
six_months_ago = datetime.utcnow() - timedelta(days=30*6)
recent_users = list(db.users.find({"dateJoined": {"$gte": six_months_ago}}))
print(recent_users)

[{'_id': ObjectId('68e1c6332cb9762fe4258ddf'), 'userId': 'instr_102', 'email': 'instr_102@example.com', 'firstName': 'David', 'lastName': 'Miller', 'role': 'instructor', 'dateJoined': datetime.datetime(2025, 9, 2, 9, 34, 18), 'profile': {'bio': 'Affect ask determine continue art decide reduce Republican kitchen social.', 'avatar': '', 'skills': ['push', 'action', 'discover']}, 'isActive': True}, {'_id': ObjectId('68e1c6332cb9762fe4258dec'), 'userId': 'stud_1010', 'email': 'stud_1010@example.com', 'firstName': 'David', 'lastName': 'Watts', 'role': 'student', 'dateJoined': datetime.datetime(2025, 5, 19, 10, 14, 21), 'profile': {'bio': 'Thus wide media Mr show score pass environment wait stock.', 'avatar': '', 'skills': ['manager', 'top']}, 'isActive': True}, {'_id': ObjectId('68e1c6332cb9762fe4258ded'), 'userId': 'stud_1011', 'email': 'stud_1011@example.com', 'firstName': 'Jennifer', 'lastName': 'Gilbert', 'role': 'student', 'dateJoined': datetime.datetime(2025, 7, 23, 16, 4, 29), 'profi

C:\Users\SYLVESTER\AppData\Local\Temp\ipykernel_3720\253433775.py:3: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  six_months_ago = datetime.utcnow() - timedelta(days=30*6)


In [32]:
# Find courses that have specific tags using $in
tags_search = list(db.courses.find({"tags": {"$in": ["python", "data"]}}))
print(tags_search)

[]


In [37]:
# Retrieve assignments due in the next week
one_week = datetime.utcnow() + timedelta(days=7)
upcoming_assignments = list(db.assignments.find({"dueDate": {"$lte": one_week, "$gte": datetime.utcnow()}}))
upcoming_assignments

C:\Users\SYLVESTER\AppData\Local\Temp\ipykernel_3720\3386083103.py:2: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  one_week = datetime.utcnow() + timedelta(days=7)
C:\Users\SYLVESTER\AppData\Local\Temp\ipykernel_3720\3386083103.py:3: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  upcoming_assignments = list(db.assignments.find({"dueDate": {"$lte": one_week, "$gte": datetime.utcnow()}}))


[{'_id': ObjectId('68e1c6332cb9762fe4258e3f'),
  'assignmentId': 'assign_2',
  'courseId': 'course_201',
  'title': 'Whole about wife for Assignment 2',
  'description': 'Former play agent.',
  'dueDate': datetime.datetime(2025, 10, 5, 10, 47, 23),
  'maxPoints': 100}]

In [36]:
# Count total enrollments per course, average course rating, group by category
pipeline = [
    {"$group": {"_id": "$courseId", "totalEnrollments": {"$sum": 1}}},
    {"$lookup": {"from":"courses","localField":"_id","foreignField":"courseId","as":"course"}},
    {"$unwind":"$course"},
    {"$project": {"courseId":"$_id","title":"$course.title","category":"$course.category","totalEnrollments":1,"avgRating":"$course.rating"}}
]
enrollment_stats = list(db.enrollments.aggregate(pipeline))

enrollment_stats


[{'_id': 'course_203',
  'totalEnrollments': 2,
  'courseId': 'course_203',
  'title': 'Health probably major',
  'category': 'business',
  'avgRating': 4.3},
 {'_id': 'course_201',
  'totalEnrollments': 6,
  'courseId': 'course_201',
  'title': 'Whole about wife for',
  'category': 'language',
  'avgRating': 4.5},
 {'_id': 'course_207',
  'totalEnrollments': 4,
  'courseId': 'course_207',
  'title': 'Remember song',
  'category': 'programming',
  'avgRating': 3.3},
 {'_id': 'course_200',
  'totalEnrollments': 6,
  'courseId': 'course_200',
  'title': 'By woman success organization speech',
  'category': 'finance',
  'avgRating': 3.7},
 {'_id': 'course_204',
  'totalEnrollments': 4,
  'courseId': 'course_204',
  'title': 'Player author great',
  'category': 'finance',
  'avgRating': 4.0},
 {'_id': 'course_202',
  'totalEnrollments': 6,
  'courseId': 'course_202',
  'title': 'He measure',
  'category': 'design',
  'avgRating': 3.6},
 {'_id': 'course_206',
  'totalEnrollments': 1,
  'cou

In [39]:
pipeline = [
    {"$group": {"_id": "$studentId","avgGrade": {"$avg":"$grade"}, "submissionCount": {"$sum":1}}},
    {"$sort": {"avgGrade": -1}}
]
avg_grade_per_student = list(db.submissions.aggregate(pipeline))

avg_grade_per_student

[{'_id': 'stud_1006', 'avgGrade': 94.3, 'submissionCount': 1},
 {'_id': 'stud_1014', 'avgGrade': 91.2, 'submissionCount': 1},
 {'_id': 'stud_1010', 'avgGrade': 90.05, 'submissionCount': 2},
 {'_id': 'stud_1002', 'avgGrade': 87.65, 'submissionCount': 2},
 {'_id': 'stud_1012', 'avgGrade': 80.85, 'submissionCount': 2},
 {'_id': 'stud_1011', 'avgGrade': 79.05, 'submissionCount': 2},
 {'_id': 'stud_1004', 'avgGrade': 77.0, 'submissionCount': 1},
 {'_id': 'stud_1000', 'avgGrade': 68.5, 'submissionCount': 1},
 {'_id': 'stud_1009', 'avgGrade': 62.5, 'submissionCount': 2},
 {'_id': 'stud_1008', 'avgGrade': 50.7, 'submissionCount': 1}]

In [40]:
# Completion rate by course (completed enrollments / total enrollments)
pipeline = [
    {"$group": {"_id": {"courseId":"$courseId", "status":"$status"}, "count": {"$sum":1}}},
    {"$group": {"_id":"$_id.courseId", "counts":{"$push":{"status":"$_id.status","count":"$count"}}, "total":{"$sum":"$count"}}},
    {"$project": {
        "courseId":"$_id",
        "completed": {"$filter":{"input":"$counts","as":"c","cond":{"$eq":["$$c.status","completed"]}}},
        "total":"$total"
    }},
    {"$unwind": {"path":"$completed","preserveNullAndEmptyArrays":True}},
    {"$project": {"courseId":1, "completionRate":{"$cond":[{"$gt":["$total",0]}, {"$multiply":[{"$divide":["$completed.count","$total"]},100]}, 0]}}}
]
completion_rates = list(db.enrollments.aggregate(pipeline))

completion_rates

[{'_id': 'course_201',
  'courseId': 'course_201',
  'completionRate': 33.33333333333333},
 {'_id': 'course_200',
  'courseId': 'course_200',
  'completionRate': 33.33333333333333},
 {'_id': 'course_203', 'courseId': 'course_203', 'completionRate': 50.0},
 {'_id': 'course_206', 'courseId': 'course_206', 'completionRate': 100.0},
 {'_id': 'course_204', 'courseId': 'course_204', 'completionRate': 25.0},
 {'_id': 'course_202',
  'courseId': 'course_202',
  'completionRate': 16.666666666666664},
 {'_id': 'course_207', 'courseId': 'course_207', 'completionRate': 100.0},
 {'_id': 'course_205', 'courseId': 'course_205', 'completionRate': 50.0}]

In [41]:
# Revenue per instructor (sum of course price * enrollments)
pipeline = [
    {"$lookup": {"from":"courses","localField":"courseId","foreignField":"courseId","as":"course"}},
    {"$unwind":"$course"},
    {"$group": {"_id":"$course.instructorId", "revenue":{"$sum":"$course.price"}, "students":{"$addToSet":"$studentId"}}},
    {"$project": {"instructorId":"$_id","revenue":1,"totalStudents":{"$size":"$students"}}}
]
instructor_revenue = list(db.enrollments.aggregate(pipeline))

instructor_revenue

[{'_id': 'instr_104',
  'revenue': 0,
  'instructorId': 'instr_104',
  'totalStudents': 6},
 {'_id': 'instr_103',
  'revenue': 399.96,
  'instructorId': 'instr_103',
  'totalStudents': 4},
 {'_id': 'instr_100',
  'revenue': 2039.88,
  'instructorId': 'instr_100',
  'totalStudents': 9},
 {'_id': 'instr_101',
  'revenue': 319.93,
  'instructorId': 'instr_101',
  'totalStudents': 7},
 {'_id': 'instr_102',
  'revenue': 799.96,
  'instructorId': 'instr_102',
  'totalStudents': 4}]

In [42]:
pipeline = [
    {"$project": {"year": {"$year":"$enrolledAt"}, "month":{"$month":"$enrolledAt"}}},
    {"$group": {"_id":{"year":"$year","month":"$month"}, "count":{"$sum":1}}},
    {"$sort": {"_id.year":1, "_id.month":1}}
]
monthly_trends = list(db.enrollments.aggregate(pipeline))

monthly_trends


[{'_id': {'year': 2024, 'month': 4}, 'count': 1},
 {'_id': {'year': 2024, 'month': 5}, 'count': 1},
 {'_id': {'year': 2024, 'month': 6}, 'count': 3},
 {'_id': {'year': 2024, 'month': 7}, 'count': 1},
 {'_id': {'year': 2024, 'month': 8}, 'count': 2},
 {'_id': {'year': 2024, 'month': 12}, 'count': 1},
 {'_id': {'year': 2025, 'month': 2}, 'count': 4},
 {'_id': {'year': 2025, 'month': 3}, 'count': 1},
 {'_id': {'year': 2025, 'month': 4}, 'count': 1},
 {'_id': {'year': 2025, 'month': 5}, 'count': 2},
 {'_id': {'year': 2025, 'month': 6}, 'count': 1},
 {'_id': {'year': 2025, 'month': 7}, 'count': 2},
 {'_id': {'year': 2025, 'month': 8}, 'count': 3},
 {'_id': {'year': 2025, 'month': 9}, 'count': 9},
 {'_id': {'year': 2025, 'month': 10}, 'count': 1}]

In [46]:
# Create indexes
db.users.create_index("email", unique=True, name="idx_users_email")
db.users.create_index([("userId", 1)], unique=True, name="idx_users_userid")
db.courses.create_index([("title", "text"), ("description","text")], name="idx_courses_text")  # text search
db.courses.create_index([("category", 1)], name="idx_courses_category")
db.courses.create_index([("courseId",1)], unique=True, name="idx_courseid")
db.assignments.create_index([("dueDate", 1)], name="idx_assign_dueDate")
db.enrollments.create_index([("studentId",1)], name="idx_enr_student")
db.enrollments.create_index([("courseId",1)], name="idx_enr_course")


'idx_enr_course'

In [44]:
pipeline = [
    {"$group": {"_id": "$email", "count": {"$sum": 1}, "ids": {"$push": "$_id"}}},
    {"$match": {"count": {"$gt": 1}}}
]
duplicates = list(db.users.aggregate(pipeline))
for dup in duplicates:
    print(dup)

{'_id': 'newstudent@example.com', 'count': 2, 'ids': [ObjectId('68e1c6842cb9762fe4258e53'), ObjectId('68e1c70a2cb9762fe4258e56')]}


In [45]:
for dup in duplicates:
    # Keep first document, remove others
    for _id in dup["ids"][1:]:
        db.users.delete_one({"_id": _id})


In [47]:
import time
q = {"courseId":"course_999"}
start = time.time()
res = db.enrollments.find(q).explain()
elapsed = time.time() - start
print("Explain:", res.get('queryPlanner', {}))
print("Elapsed (s):", elapsed)

Explain: {'namespace': 'eduhub_db.enrollments', 'parsedQuery': {'courseId': {'$eq': 'course_999'}}, 'indexFilterSet': False, 'queryHash': '5644C0BC', 'planCacheShapeHash': '5644C0BC', 'planCacheKey': '9FAD8D3B', 'optimizationTimeMillis': 13, 'maxIndexedOrSolutionsReached': False, 'maxIndexedAndSolutionsReached': False, 'maxScansToExplodeReached': False, 'prunedSimilarIndexes': False, 'winningPlan': {'isCached': False, 'stage': 'FETCH', 'inputStage': {'stage': 'IXSCAN', 'keyPattern': {'courseId': 1}, 'indexName': 'idx_enr_course', 'isMultiKey': False, 'multiKeyPaths': {'courseId': []}, 'isUnique': False, 'isSparse': False, 'isPartial': False, 'indexVersion': 2, 'direction': 'forward', 'indexBounds': {'courseId': ['["course_999", "course_999"]']}}}, 'rejectedPlans': []}
Elapsed (s): 0.10388565063476562


In [48]:

from pymongo.errors import DuplicateKeyError, WriteError

# Duplicate key handling example
try:
    db.users.insert_one(new_user)  # might violate unique email/userId
except DuplicateKeyError as e:
    print("Duplicate key:", e.details or str(e))

# Invalid data type insertion (violates schema)
invalid_user = {"userId":"u_bad", "email":"not-an-email", "firstName": 123, "lastName":"X", "role":"student", "dateJoined": datetime.utcnow(), "isActive": True}
try:
    db.users.insert_one(invalid_user)
except WriteError as e:
    print("WriteError (validation failure):", e.details or str(e))

Duplicate key: {'index': 0, 'code': 11000, 'errmsg': 'E11000 duplicate key error collection: eduhub_db.users index: idx_users_email dup key: { email: "newstudent@example.com" }', 'keyPattern': {'email': 1}, 'keyValue': {'email': 'newstudent@example.com'}}
WriteError (validation failure): {'index': 0, 'code': 121, 'errmsg': 'Document failed validation', 'errInfo': {'failingDocumentId': ObjectId('68e21ff82cb9762fe4258e61'), 'details': {'operatorName': '$jsonSchema', 'schemaRulesNotSatisfied': [{'operatorName': 'properties', 'propertiesNotSatisfied': [{'propertyName': 'email', 'details': [{'operatorName': 'pattern', 'specifiedAs': {'pattern': '^[^@ \\t\\r\\n]+@[^@ \\t\\r\\n]+\\.[^@ \\t\\r\\n]+$'}, 'reason': 'regular expression did not match', 'consideredValue': 'not-an-email'}]}, {'propertyName': 'firstName', 'details': [{'operatorName': 'bsonType', 'specifiedAs': {'bsonType': 'string'}, 'reason': 'type did not match', 'consideredValue': 123, 'consideredType': 'int'}]}]}]}}}


C:\Users\SYLVESTER\AppData\Local\Temp\ipykernel_3720\1960855169.py:10: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  invalid_user = {"userId":"u_bad", "email":"not-an-email", "firstName": 123, "lastName":"X", "role":"student", "dateJoined": datetime.utcnow(), "isActive": True}


In [51]:
import json
collections = ["users","courses","enrollments","lessons","assignments","submissions"]
export = {}
for c in collections:
    docs = list(db[c].find({}, {"_id":0}))  # drop _id for portability
    export[c] = docs

with open("data/sample_data.json","w", encoding="utf-8") as f:
    json.dump(export, f, default=str, indent=2)
print("Exported sample_data.json")

Exported sample_data.json


In [50]:
import os, json

# Ensure folder exists
os.makedirs("data", exist_ok=True)

# Now export safely
export = {}
for c in db.list_collection_names():
    docs = list(db[c].find({}, {"_id": 0}))
    export[c] = docs

with open("data/sample_data.json", "w", encoding="utf-8") as f:
    json.dump(export, f, default=str, indent=2)

print("✅ Exported data/sample_data.json successfully.")


✅ Exported data/sample_data.json successfully.
